In [1]:
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import linregress, spearmanr, ttest_ind
from math import ceil
import MolecularClocks.src.methylation_util as m_util
from MolecularClocks.src.merge_pdfs import merge_pdfs
from MolecularClocks.src.util import combineFilters
from MolecularClocks.src.invasiveCpGs_consts import getConsts
# from expandAnnotations import expand

consts = getConsts()

sns.set(rc={"savefig.bbox":'tight'}, font_scale=1.2)

/var/folders/7j/2z93zxs52qn4yjl8szt8gys80000hf/T/ipykernel_60121/1266859012.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
TCGA_datadir = os.path.join(consts['official_indir'], 'TCGA')

In [3]:
TCGA_methyl = pd.read_table(os.path.join(TCGA_datadir, 'cohort1.methyl.tsv'), index_col=0)

In [4]:
# purityEstimates = consts['purityEstimates']()
purityEstimates = pd.read_csv(os.path.join(consts['official_indir'], 'TCGA', 'ncomms9971-s2.csv'), index_col=0)

In [7]:
data = {'tumor':{}, 'normal':{}}

In [8]:
data['tumor']['beta_values'] = TCGA_methyl
data['normal']['beta_values'] = consts['project_data_func_dict']['Johnson']['all']()

# sample_annotations = pd.read_table(os.path.join(consts['indir'], 'ALL', 'clinical_cleaned_processed.txt'), index_col=0)
# cleanAnnot.

In [23]:
patientIDfromSampleID = lambda x:'-'.join(x.split('-')[-1])

In [55]:
data['tumor']['purity_threshold'] = consts['CPE_threshold']
data['normal']['purity_threshold'] = consts['lump_threshold']('Johnson')

In [46]:
shortenSampleIDs = lambda x:'-'.join(x.split('-')[:4])
data['tumor']['beta_values'] = data['tumor']['beta_values'].rename(shortenSampleIDs, axis='columns')

In [47]:
data['tumor']['beta_values'] = data['tumor']['beta_values'].loc[:, ~data['tumor']['beta_values'].columns.duplicated()]
data['tumor']['beta_values'] = data['tumor']['beta_values'].loc[:, data['tumor']['beta_values'].columns.isin(purityEstimates.index)]

data['tumor']['purity'] = purityEstimates.loc[data['tumor']['beta_values'].columns, 'CPE']
data['normal']['purity'] = m_util.getLUMP_values(data['normal']['beta_values'])

In [56]:
for sample in ['tumor', 'normal']:
    data[sample]['pureSamples'] = data[sample]['purity'].index[data[sample]['purity'] > data[sample]['purity_threshold']].values
    data[sample]['beta_values_PURE'] = data[sample]['beta_values'][data[sample]['pureSamples']]

    data[sample]['beta_means'] = data[sample]['beta_values_PURE'].mean(axis=1)
    data[sample]['beta_stds'] = data[sample]['beta_values_PURE'].std(axis=1)
    data[sample]['beta_nans'] = data[sample]['beta_values_PURE'].isna().sum(axis=1)

In [66]:
chip450_info = pd.read_table(os.path.join(consts['official_indir'], 'TCGA', 'chip450_annot_cleaned.txt'), index_col=0)
chip850_info = pd.read_table(os.path.join(consts['official_indir'], 'TCGA', 'chip850_annot_cleaned.txt'), index_col=0)

/var/folders/7j/2z93zxs52qn4yjl8szt8gys80000hf/T/ipykernel_60121/420267308.py:1: DtypeWarning: Columns (5,7,8,14,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  chip450_info = pd.read_table(os.path.join(consts['official_indir'], 'TCGA', 'chip450_annot_cleaned.txt'), index_col=0)
/var/folders/7j/2z93zxs52qn4yjl8szt8gys80000hf/T/ipykernel_60121/420267308.py:2: DtypeWarning: Columns (5,6,7,8,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  chip850_info = pd.read_table(os.path.join(consts['official_indir'], 'TCGA', 'chip850_annot_cleaned.txt'), index_col=0)


In [67]:
neutral_filters = [
    (chip450_info['Regulatory_Feature_Group']=='None'),
    ~chip450_info['HAS_geneName'],
    chip850_info['Regulatory_Feature_Group'].isna(),
    chip850_info['UCSC_RefGene_Name'].isna(),
]

neutral_DNA_CpG_list = chip450_info.index[combineFilters(neutral_filters)].values

## Sanity checks - IMPORTANT - experienced weird errors before with selecting CpGs that did not fit the criteria...
neutral_chip450_info = chip450_info.loc[neutral_DNA_CpG_list]
neutral_chip850_info = chip850_info.loc[neutral_DNA_CpG_list]
assert (neutral_chip450_info['Regulatory_Feature_Group']=='None').all()
assert (~neutral_chip450_info['HAS_geneName']).all()
assert neutral_chip850_info['Regulatory_Feature_Group'].isna().all()
assert neutral_chip850_info['UCSC_RefGene_Name'].isna().all()

In [ ]:
balanced_CpGs = m_util.getCpG_list(data, balanced_criteria, starting_CpG_list=neutral_DNA_CpG_list, n_select=500)